<a href="https://colab.research.google.com/github/candiceevemiller/personal-projects/blob/main/Wordle_Cheat_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
"""
A program that takes your wordle clues and gives you potential matches from the
official wordle answer bank
"""

import re

In [35]:
# Loads wordle answers and stores in list
!wget https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt

wordle_answers = []
with open('WordleAnswers.txt', 'r') as f:
    for line in f:
        for word in line[1:-1].split(','):
            wordle_answers.append(word.strip('"'))
wordle_answers.sort()
print(f'Number of possible answers: {len(wordle_answers)}\n')
wordle_answers[:25]

--2022-02-03 04:41:19--  https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18521 (18K) [text/plain]
Saving to: ‘WordleAnswers.txt.5’

WordleAnswers.txt.5 100%[===================>]  18.09K  --.-KB/s    in 0.001s  

2022-02-03 04:41:19 (24.4 MB/s) - ‘WordleAnswers.txt.5’ saved [18521/18521]

Number of possible answers: 2315



['aback',
 'abase',
 'abate',
 'abbey',
 'abbot',
 'abhor',
 'abide',
 'abled',
 'abode',
 'abort',
 'about',
 'above',
 'abuse',
 'abyss',
 'acorn',
 'acrid',
 'actor',
 'acute',
 'adage',
 'adapt',
 'adept',
 'admin',
 'admit',
 'adobe',
 'adopt']

In [26]:
# TODO: Refactor to take lists instead of dict for green and yellow
#       update docstring and type hints to match
# TODO: Refactor to remove repr, raw string unnecessary, needlessly complicated
# TODO: Refactor control flows to better use filters & generators

def potential_answers(green: dict, yellow: dict, gray: str) -> list:
    """
    Takes 2 dicts, green, yellow, and a string gray
    example expected format:
    green = {1:'a', 2:'', 3:'', 4:'', 5:'e'}
    yellow = {1:'', 2:'ok', 3:'', 4: '', 5:''}
    gray = 'brlm'
    """

    search_sequence = ['.' for i in range(5)]  # create list equivalent to string '.....'

    # iterate over green dict and if the value isn't empty replace the character
    # at the matching index in search_sequence
    for key, val in green.items():
        if val:
            search_sequence[key-1] = val

    # create an empty string to store letters we know are in the word but don't know where
    # then we iterate over yellow dict and create a regex NOT term for each
    # non-empty index and append the relavent terms to the in_word variable
    # this way we know to look for words that have a letter but not at the Xth index
    in_word = ''
    for key, val in yellow.items():
        in_word += val
        if val and search_sequence[key-1] == '.':
            search_sequence[key-1] = '[^' + val + ']'

    # converts search_sequence list into a string then into a raw string and slices
    # off the single quotes introduced through the repr function
    search_sequence = repr(''.join(search_sequence))[1:-1]
    r = re.compile(search_sequence)

    # filters based on green known letters and which letters aren't at an index
    # but doesn't enforce the presence of yellow letters
    matches = list(filter(r.match, wordle_answers))

    # iterate through each word in matches, then iterates through every letter
    # we know must be in the word, if all letters are in the word we append the
    # word to our matches2 list
    matches2 = []
    for word in matches:
        flag = True
        for letter in in_word:
            if letter not in word:
                flag = False
        if flag:
            matches2.append(word)

    # iterate through every word in our matches2 list and then iterate through
    # every letter we know NOT to be in the word. If it contains none of the gray
    # letters we append our word to our final output list
    final_matches = []
    for word in matches2:
        flag = True
        for letter in gray:
            if letter in word:
                flag = False
        if flag:
            final_matches.append(word)
            
    print('\nYour potential matches are:\n')
    return final_matches

In [27]:
# Test case with actual guesses for wordle 227
green = {1:'', 2:'', 3:'', 4:'', 5:'e'}
yellow = {1:'s', 2:'et', 3:'', 4: '', 5:''}
gray = 'paryl'
potential_answers(green, yellow, gray)


Your potential matches are:



['these', 'those']

In [28]:
# TODO: Refactor to return list instead of dict

def get_green() -> dict:
    """
    Gets and validates user input for green letters and formats for feeding into
    the function potential_answers()
    """

    green = {1:'', 2:'', 3:'', 4:'', 5:''}  # initialize green dict

    print("\nLet's get your green letters:")
    print("-----------------------------")

    for i in range(1, 6):
        flag = ''  # helps us validate user, keeping us in the while loop if not y/n        

        while flag != 'y' and flag != 'n':
            flag = input(f'Do you know what letter is in spot {i}? (y/n)\n')
            if flag != 'y' and flag !='n':
                print('Please only type y or n right now\n')

        if flag == 'y':
            letter = ''
            while not letter.isalpha() or len(letter) != 1:
                letter = input(f'What letter do you have for spot {i}?\n')
                if not letter.isalpha() or len(letter) != 1:
                    print('Please only type a letter right now')
            # store standardized user input in the correct index of green dict
            green[i] = letter.lower()

    return green

In [29]:
# TODO: Refactor to return list instead of dict
# TODO: Pass green to yellow to remove needless user entry

def get_yellow(green: list) -> dict:
    """
    Gets and validates user input for yellow letters and formats for passing into
    function potential_answers()
    """

    yellow = {1:'', 2:'', 3:'', 4:'', 5:''}

    print("\nLet's get your yellow letters:")
    print("-----------------------------")

    for i in range(1, 6):
        flag = 0 # again helps us validate user input

        while flag != 'y' and flag != 'n':
            flag = input(f'Do you know a yellow letter for spot {i}? (y/n)\n')
            if flag != 'y' and flag !='n':
                print('Please only type y or n right now\n')

        if flag == 'y':
            letters = ''
            while not letters.isalpha():
                letters = input(f'What yellow letters do you know for spot {i}?\n(Type all that apply)\n')
                if not letters.isalpha():
                    print('Please only type a letter right now')
            # stores standardized user input in correct index of yellow dict
            yellow[i] = letters.lower()
            
    return yellow

In [30]:
# TODO: get_gray() doesn't currently accept no letters.
# allow get_gray() to return ''

def get_gray() -> str:
    """
    Gets and validates user input for gray letters and formats for passing into
    function potential_answers
    """

    gray = ''

    print(f"\nLet's get your gray letters:")
    print('-----------------------------')

    while not gray and not gray.isalpha():
        gray = input(f'What are your gray letters?\n(Type all that apply)\n')
        if not gray.isalpha() and gray != '':
            print('Please only type letters')
            
    return gray.lower()

In [31]:
def get_answers():
    """Combines input functions and output function into one"""

    green = get_green()
    yellow = get_yellow(green)
    gray = get_gray()

    return potential_answers(green, yellow, gray)


Let's get your green letters:
-----------------------------
Do you know what letter is in spot 1? (y/n)
y
What letter do you have for spot 1?
s
Do you know what letter is in spot 2? (y/n)
n
Do you know what letter is in spot 3? (y/n)
n
Do you know what letter is in spot 4? (y/n)
y
What letter do you have for spot 4?
t
Do you know what letter is in spot 5? (y/n)
n

Let's get your yellow letters:
-----------------------------
Do you know a yellow letter for spot 1? (y/n)
n
Do you know a yellow letter for spot 2? (y/n)
n
Do you know a yellow letter for spot 3? (y/n)
y
What yellow letters do you know for spot 3?
(Type all that apply)
y
Do you know a yellow letter for spot 4? (y/n)
n
Do you know a yellow letter for spot 5? (y/n)
n

Let's get your gray letters:
-----------------------------
What are your gray letters?
(Type all that apply)
q

Your potential matches are:



['salty', 'sixty', 'sooty']

In [ ]:
get_answers()